**Задание 1: Реализация обработки массива на CPU с использованием OpenMP**
1. Создайте массив данных размером `N` (например, `N = 1 000 000`).
2. Реализуйте функцию для обработки массива на CPU с использованием
OpenMP. Например, умножьте каждый элемент массива на 2.
3. Замерьте время выполнения обработки на CPU.

Описание кода:
* реализована обработка массива на CPU с OpenMP

* реализована обработка массива на GPU с CUDA

* реализован гибридный подход

* проведены замеры времени

In [1]:
%%writefile task1.cu
#include <cstdio>                                                           // printf
#include <vector>                                                           // std::vector
#include <random>                                                           // генератор случайных чисел
#include <chrono>                                                           // таймер CPU

#ifdef _OPENMP                                                               // если OpenMP доступен
#include <omp.h>                                                             // функции OpenMP
#endif

static void fill_random(std::vector<int>& a, int seed = 123)                 // Заполнение массива случайными числами
{                                                                            // начало функции
    std::mt19937 rng(seed);                                                  // генератор
    std::uniform_int_distribution<int> dist(0, 9);                           // значения 0..9
    for (size_t i = 0; i < a.size(); ++i) a[i] = dist(rng);                  // заполняем
}                                                                            // конец функции

static void cpu_process_openmp(int* a, int n)                                // CPU обработка: умножить каждый элемент на 2
{                                                                            // начало функции
    #pragma omp parallel for schedule(static)                                // OpenMP параллельный цикл
    for (int i = 0; i < n; ++i)                                              // цикл по элементам
    {                                                                        // начало цикла
        a[i] = a[i] * 2;                                                     // умножаем на 2
    }                                                                        // конец цикла
}                                                                            // конец функции

int main()                                                                    // точка входа
{                                                                             // начало main
    int N = 1'000'000;                                                        // размер массива
    std::vector<int> a(N);                                                    // массив данных
    fill_random(a);                                                           // заполняем

#ifdef _OPENMP
    printf("OpenMP включен, потоков (макс): %d\n", omp_get_max_threads());     // печатаем число потоков
#else
    printf("OpenMP не включен (компилируй с -Xcompiler -fopenmp)\n");          // предупреждение
#endif

    auto t0 = std::chrono::high_resolution_clock::now();                      // старт таймера
    cpu_process_openmp(a.data(), N);                                           // обработка массива
    auto t1 = std::chrono::high_resolution_clock::now();                      // стоп таймера

    double ms = std::chrono::duration<double, std::milli>(t1 - t0).count();   // время в мс
    printf("CPU(OpenMP) N=%d | time=%.3f ms\n", N, ms);                        // вывод времени

    // простая проверка (первые 5 элементов)
    printf("Пример (первые 5): ");
    for (int i = 0; i < 5; ++i) printf("%d ", a[i]);
    printf("\n");

    return 0;                                                                  // успешный выход
}                                                                             // конец main


Writing task1.cu


In [2]:
!nvcc -O3 -std=c++17 -Xcompiler -fopenmp task1.cu -o task1
!./task1

OpenMP включен, потоков (макс): 2
CPU(OpenMP) N=1000000 | time=0.252 ms
Пример (первые 5): 12 14 4 8 4 


**Анализ результатов:**

По результатам выполнения первого задания обработка массива размером 1 000 000 элементов на CPU с использованием OpenMP была выполнена корректно: каждый элемент был умножен на 2, что видно по примеру первых значений (они стали чётными). Время выполнения составило 0.252 мс при использовании 2 потоков, что показывает, что параллелизация на CPU уменьшает время обработки по сравнению с последовательным циклом, однако общий выигрыш ограничен небольшим числом доступных потоков и пропускной способностью памяти.

**Задание 2: Реализация обработки массива на GPU с использованием CUDA**
1. Скопируйте массив данных на GPU.
2. Реализуйте ядро CUDA для обработки массива на GPU. Например, умножьте
каждый элемент массива на 2.
3. Скопируйте обработанные данные обратно на CPU.
4. Замерьте время выполнения обработки на GPU.

In [3]:
%%writefile task2.cu
#include <cuda_runtime.h>                                                   // CUDA runtime
#include <cstdio>                                                           // printf
#include <vector>                                                           // std::vector
#include <random>                                                           // rng
#include <chrono>                                                           // CPU таймер (если нужно)
#include <cstdlib>                                                          // exit

static inline void cuda_check(cudaError_t err, const char* file, int line)  // проверка CUDA ошибок
{
    if (err != cudaSuccess)
    {
        printf("Ошибка CUDA %s:%d: %s\n", file, line, cudaGetErrorString(err));
        std::exit(1);
    }
}
#define CHECK_CUDA(call) cuda_check((call), __FILE__, __LINE__)

static void fill_random(std::vector<int>& a, int seed = 123)                // заполнение случайными
{
    std::mt19937 rng(seed);
    std::uniform_int_distribution<int> dist(0, 9);
    for (size_t i = 0; i < a.size(); ++i) a[i] = dist(rng);
}

__global__ void gpu_process_kernel(int* a, int n)                           // ядро: умножить на 2
{
    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    if (gid < n) a[gid] = a[gid] * 2;
}

int main()
{
    int N = 1'000'000;                                                     // размер массива
    int threads = 256;                                                     // потоки в блоке
    int blocks = (N + threads - 1) / threads;                              // блоки

    std::vector<int> h(N);                                                 // массив на CPU
    fill_random(h);                                                        // заполняем

    int* d = nullptr;                                                      // указатель на GPU
    CHECK_CUDA(cudaMalloc(&d, N * sizeof(int)));                            // выделяем на GPU

    cudaEvent_t start_total, stop_total, start_k, stop_k;                   // события для времени
    CHECK_CUDA(cudaEventCreate(&start_total));
    CHECK_CUDA(cudaEventCreate(&stop_total));
    CHECK_CUDA(cudaEventCreate(&start_k));
    CHECK_CUDA(cudaEventCreate(&stop_k));

    CHECK_CUDA(cudaEventRecord(start_total));                               // старт total

    CHECK_CUDA(cudaMemcpy(d, h.data(), N * sizeof(int), cudaMemcpyHostToDevice)); // H2D

    CHECK_CUDA(cudaEventRecord(start_k));                                   // старт kernel
    gpu_process_kernel<<<blocks, threads>>>(d, N);                          // запуск ядра
    CHECK_CUDA(cudaGetLastError());                                         // проверка запуска
    CHECK_CUDA(cudaEventRecord(stop_k));                                    // стоп kernel

    CHECK_CUDA(cudaMemcpy(h.data(), d, N * sizeof(int), cudaMemcpyDeviceToHost)); // D2H

    CHECK_CUDA(cudaEventRecord(stop_total));                                // стоп total
    CHECK_CUDA(cudaEventSynchronize(stop_total));                           // ждём

    float kernel_ms = 0.0f, total_ms = 0.0f;                                // времена
    CHECK_CUDA(cudaEventElapsedTime(&kernel_ms, start_k, stop_k));          // kernel time
    CHECK_CUDA(cudaEventElapsedTime(&total_ms, start_total, stop_total));   // total time

    printf("GPU(CUDA) N=%d | kernel=%.3f ms | total(H2D+kernel+D2H)=%.3f ms\n", N, kernel_ms, total_ms);

    printf("Пример (первые 5): ");
    for (int i = 0; i < 5; ++i) printf("%d ", h[i]);
    printf("\n");

    CHECK_CUDA(cudaEventDestroy(start_total));
    CHECK_CUDA(cudaEventDestroy(stop_total));
    CHECK_CUDA(cudaEventDestroy(start_k));
    CHECK_CUDA(cudaEventDestroy(stop_k));
    CHECK_CUDA(cudaFree(d));

    return 0;
}

Writing task2.cu


In [5]:
!nvcc -O3 -std=c++17 task2.cu -o task2 -gencode arch=compute_75,code=sm_75
!./task2

GPU(CUDA) N=1000000 | kernel=0.073 ms | total(H2D+kernel+D2H)=2.062 ms
Пример (первые 5): 12 14 4 8 4 


**Анализ результатов:**

По результатам второго задания обработка массива на GPU выполнена корректно, что подтверждается совпадением примера первых элементов с ожидаемым результатом (значения также стали чётными после умножения на 2). Время выполнения ядра составило 0.073 мс, то есть сама вычислительная часть на GPU заметно быстрее CPU-обработки из первого задания. Однако общее время выполнения 2.062 мс оказалось значительно больше kernel-времени, потому что сюда входят накладные расходы на копирование данных между CPU и GPU (H2D и D2H) и синхронизация. Это показывает, что при единичной операции “умножить на 2” главным ограничением становится не вычисление, а передача данных.

**Задание 3: Гибридная обработка массива**
1. Разделите массив на две части: первая половина обрабатывается на CPU,
вторая — на GPU.
2. Реализуйте гибридное приложение, которое выполняет обработку массива
на CPU и GPU одновременно.
3. Замерьте общее время выполнения гибридной обработки.

In [6]:
%%writefile task3.cu
#include <cuda_runtime.h>                                                   // CUDA runtime
#include <cstdio>                                                           // printf
#include <cstdlib>                                                          // exit
#include <random>                                                           // rng
#include <chrono>                                                           // таймер
#include <thread>                                                           // std::thread

#ifdef _OPENMP
#include <omp.h>
#endif

static inline void cuda_check(cudaError_t err, const char* file, int line)  // проверка CUDA
{
    if (err != cudaSuccess)
    {
        printf("Ошибка CUDA %s:%d: %s\n", file, line, cudaGetErrorString(err));
        std::exit(1);
    }
}
#define CHECK_CUDA(call) cuda_check((call), __FILE__, __LINE__)

__global__ void gpu_process_kernel(int* a, int n)                           // ядро: умножение на 2
{
    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    if (gid < n) a[gid] = a[gid] * 2;
}

static void cpu_process_openmp(int* a, int n)                               // CPU обработка (OpenMP)
{
    #pragma omp parallel for schedule(static)
    for (int i = 0; i < n; ++i) a[i] = a[i] * 2;
}

static void fill_random(int* a, int n, int seed = 123)                      // заполнение массива
{
    std::mt19937 rng(seed);
    std::uniform_int_distribution<int> dist(0, 9);
    for (int i = 0; i < n; ++i) a[i] = dist(rng);
}

int main()
{
    int N = 1'000'000;                                                     // размер массива
    int threads = 256;                                                     // потоки CUDA

#ifdef _OPENMP
    printf("OpenMP включен, потоков (макс): %d\n", omp_get_max_threads());
#else
    printf("OpenMP не включен (компилируй с -Xcompiler -fopenmp)\n");
#endif

    // Важно: используем pinned память, чтобы cudaMemcpyAsync была реально асинхронной
    int* h = nullptr;                                                      // массив на CPU (pinned)
    CHECK_CUDA(cudaHostAlloc(&h, N * sizeof(int), cudaHostAllocDefault));   // выделяем pinned

    fill_random(h, N, 123);                                                // заполняем

    int n1 = N / 2;                                                        // первая половина
    int n2 = N - n1;                                                       // вторая половина
    int offset = n1;                                                       // смещение второй половины

    cudaStream_t stream;                                                   // CUDA stream
    CHECK_CUDA(cudaStreamCreate(&stream));                                 // создаём stream

    auto t0 = std::chrono::high_resolution_clock::now();                   // старт общего времени

    float gpu_kernel_ms = 0.0f;                                            // kernel время GPU половины (для справки)

    // GPU часть запускаем в отдельном потоке CPU
    std::thread gpu_thread([&]() {
        int* d = nullptr;                                                  // память под половину на GPU
        CHECK_CUDA(cudaMalloc(&d, n2 * sizeof(int)));                       // выделяем

        int blocks = (n2 + threads - 1) / threads;                          // блоки

        cudaEvent_t ks, ke;                                                 // события kernel времени
        CHECK_CUDA(cudaEventCreate(&ks));
        CHECK_CUDA(cudaEventCreate(&ke));

        CHECK_CUDA(cudaMemcpyAsync(d, h + offset, n2 * sizeof(int), cudaMemcpyHostToDevice, stream)); // H2D async

        CHECK_CUDA(cudaEventRecord(ks, stream));                            // старт kernel
        gpu_process_kernel<<<blocks, threads, 0, stream>>>(d, n2);          // ядро
        CHECK_CUDA(cudaGetLastError());
        CHECK_CUDA(cudaEventRecord(ke, stream));                            // стоп kernel

        CHECK_CUDA(cudaMemcpyAsync(h + offset, d, n2 * sizeof(int), cudaMemcpyDeviceToHost, stream)); // D2H async
        CHECK_CUDA(cudaStreamSynchronize(stream));                          // ждём stream

        CHECK_CUDA(cudaEventElapsedTime(&gpu_kernel_ms, ks, ke));           // kernel время

        CHECK_CUDA(cudaEventDestroy(ks));
        CHECK_CUDA(cudaEventDestroy(ke));
        CHECK_CUDA(cudaFree(d));
    });

    // CPU часть выполняется одновременно
    cpu_process_openmp(h, n1);                                              // обрабатываем первую половину

    gpu_thread.join();                                                     // ждём GPU поток

    auto t1 = std::chrono::high_resolution_clock::now();                   // стоп
    double total_ms = std::chrono::duration<double, std::milli>(t1 - t0).count();

    printf("HYBRID N=%d | total=%.3f ms | gpu_half_kernel=%.3f ms (справочно)\n", N, total_ms, gpu_kernel_ms);

    printf("Пример (первые 5): ");
    for (int i = 0; i < 5; ++i) printf("%d ", h[i]);
    printf("\n");

    CHECK_CUDA(cudaStreamDestroy(stream));
    CHECK_CUDA(cudaFreeHost(h));

    return 0;
}

Writing task3.cu


In [10]:
!nvcc -O3 -std=c++17 -Xcompiler -fopenmp task3.cu -o task3 -gencode arch=compute_75,code=sm_75
!./task3

OpenMP включен, потоков (макс): 2
HYBRID N=1000000 | total=0.951 ms | gpu_half_kernel=0.060 ms (справочно)
Пример (первые 5): 12 14 4 8 4 


**Анализ результатов:**

По результатам третьего задания гибридная обработка массива размером 1 000 000 элементов выполнена корректно, что подтверждается примером первых значений после умножения на 2. Общее время гибридного режима составило 0.951 мс, при этом kernel-время GPU для второй половины массива было всего 0.060 мс, то есть вычисления на GPU выполняются быстро. Однако итоговое время гибрида оказалось выше, чем у чистого CPU в первом задании (0.252 мс), поскольку в гибридном режиме сохраняются накладные расходы на передачу данных между CPU и GPU и дополнительная стоимость организации параллельного запуска (stream, асинхронные копирования, поток). Это показывает, что при простой операции и относительно небольшом числе потоков OpenMP гибридный подход не даёт выигрыша и становится выгодным только при более тяжёлых вычислениях или при повторном использовании данных на GPU без постоянных копирований.

**Задание 4: Анализ производительности**
1. Сравните время выполнения обработки массива на CPU, GPU и в гибридном
режиме.
2. Проведите анализ производительности и определите, в каких случаях
гибридный подход дает наибольший выигрыш.

In [11]:
%%writefile task4.cu
#include <cuda_runtime.h>                 // Подключаем CUDA Runtime API для работы с GPU
#include <cstdio>                         // Подключаем printf для вывода в консоль
#include <cstdlib>                        // Подключаем std::exit для аварийного завершения программы
#include <vector>                         // Подключаем std::vector для хранения массивов на CPU
#include <random>                         // Подключаем генератор случайных чисел
#include <chrono>                         // Подключаем средства измерения времени
#include <thread>                         // Подключаем std::thread для гибридного режима

#ifdef _OPENMP
#include <omp.h>                          // Подключаем OpenMP, если он поддерживается
#endif

static inline void cuda_check(cudaError_t err, const char* file, int line)
{
    if (err != cudaSuccess)               // Если произошла ошибка CUDA
    {
        printf("Ошибка CUDA %s:%d: %s\n", // Выводим имя файла, номер строки и описание ошибки
               file, line, cudaGetErrorString(err));
        std::exit(1);                     // Завершаем программу с ошибкой
    }
}

#define CHECK_CUDA(call) cuda_check((call), __FILE__, __LINE__) // Макрос для удобной проверки CUDA-вызовов

static void fill_random(std::vector<int>& a, int seed = 123)
{
    std::mt19937 rng(seed);               // Инициализируем генератор случайных чисел
    std::uniform_int_distribution<int> dist(0, 9); // Диапазон случайных чисел от 0 до 9
    for (size_t i = 0; i < a.size(); ++i) // Проходим по всему массиву
        a[i] = dist(rng);                 // Записываем случайное значение в элемент массива
}

static void cpu_process_openmp(int* a, int n)
{
    #pragma omp parallel for schedule(static) // Параллельный цикл OpenMP
    for (int i = 0; i < n; ++i)              // Проходим по всем элементам массива
        a[i] *= 2;                           // Умножаем каждый элемент на 2
}

__global__ void gpu_process_kernel(int* a, int n)
{
    int gid = blockIdx.x * blockDim.x + threadIdx.x; // Вычисляем глобальный индекс потока
    if (gid < n)                                    // Проверяем выход за границы массива
        a[gid] *= 2;                               // Умножаем элемент массива на 2
}

static float gpu_total_ms(int* h, int n, int threads)
{
    int blocks = (n + threads - 1) / threads; // Вычисляем количество блоков CUDA

    int* d = nullptr;                          // Указатель на массив в памяти GPU
    CHECK_CUDA(cudaMalloc(&d, n * sizeof(int))); // Выделяем память на GPU

    cudaEvent_t s, e;                          // CUDA-события для замера времени
    CHECK_CUDA(cudaEventCreate(&s));           // Создаем событие начала
    CHECK_CUDA(cudaEventCreate(&e));           // Создаем событие конца

    CHECK_CUDA(cudaEventRecord(s));            // Фиксируем начало общего времени

    CHECK_CUDA(cudaMemcpy(d, h, n * sizeof(int),
                           cudaMemcpyHostToDevice)); // Копируем данные с CPU на GPU

    gpu_process_kernel<<<blocks, threads>>>(d, n);   // Запускаем CUDA-ядро
    CHECK_CUDA(cudaGetLastError());                   // Проверяем корректность запуска ядра

    CHECK_CUDA(cudaMemcpy(h, d, n * sizeof(int),
                           cudaMemcpyDeviceToHost)); // Копируем результат обратно на CPU

    CHECK_CUDA(cudaEventRecord(e));            // Фиксируем конец общего времени
    CHECK_CUDA(cudaEventSynchronize(e));       // Ждём завершения всех операций

    float ms = 0.0f;                           // Переменная для хранения времени
    CHECK_CUDA(cudaEventElapsedTime(&ms, s, e)); // Вычисляем общее время выполнения

    CHECK_CUDA(cudaEventDestroy(s));           // Удаляем событие начала
    CHECK_CUDA(cudaEventDestroy(e));           // Удаляем событие конца
    CHECK_CUDA(cudaFree(d));                   // Освобождаем память GPU

    return ms;                                 // Возвращаем общее время GPU
}

static double hybrid_total_ms(int* pinned, int n, int threads)
{
    int n1 = n / 2;                            // Размер первой половины массива
    int n2 = n - n1;                           // Размер второй половины массива
    int offset = n1;                           // Смещение для второй половины

    cudaStream_t stream;                       // CUDA stream для асинхронных операций
    CHECK_CUDA(cudaStreamCreate(&stream));     // Создаем stream

    auto t0 = std::chrono::high_resolution_clock::now(); // Старт общего таймера

    std::thread gpu_thread([&]()               // Запускаем GPU-часть в отдельном CPU-потоке
    {
        int* d = nullptr;                      // Указатель на память GPU
        CHECK_CUDA(cudaMalloc(&d, n2 * sizeof(int))); // Выделяем память под вторую половину

        int blocks = (n2 + threads - 1) / threads; // Количество блоков CUDA

        CHECK_CUDA(cudaMemcpyAsync(d, pinned + offset,
                                   n2 * sizeof(int),
                                   cudaMemcpyHostToDevice, stream)); // Асинхронная копия на GPU

        gpu_process_kernel<<<blocks, threads, 0, stream>>>(d, n2); // Запуск ядра в stream
        CHECK_CUDA(cudaGetLastError());                             // Проверка ядра

        CHECK_CUDA(cudaMemcpyAsync(pinned + offset, d,
                                   n2 * sizeof(int),
                                   cudaMemcpyDeviceToHost, stream)); // Асинхронная копия обратно

        CHECK_CUDA(cudaStreamSynchronize(stream)); // Ждём завершения GPU-операций
        CHECK_CUDA(cudaFree(d));                   // Освобождаем память GPU
    });

    cpu_process_openmp(pinned, n1);               // CPU обрабатывает первую половину массива

    gpu_thread.join();                            // Ждём завершения GPU-потока

    auto t1 = std::chrono::high_resolution_clock::now(); // Останавливаем таймер
    double ms = std::chrono::duration<double, std::milli>(t1 - t0).count(); // Считаем время

    CHECK_CUDA(cudaStreamDestroy(stream));        // Удаляем CUDA stream
    return ms;                                    // Возвращаем общее время гибридной обработки
}

int main()
{
    int N = 1'000'000;                            // Размер массива
    int threads = 256;                            // Количество потоков CUDA в блоке

#ifdef _OPENMP
    printf("OpenMP включен, потоков (макс): %d\n",
           omp_get_max_threads());                // Вывод количества потоков OpenMP
#else
    printf("OpenMP не включен\n");                // Сообщение, если OpenMP недоступен
#endif

    printf("N=%d\n\n", N);                        // Печать размера массива

    std::vector<int> base(N);                     // Исходный массив
    fill_random(base);                            // Заполняем его случайными числами

    std::vector<int> cpu = base;                  // Копия массива для CPU
    auto c0 = std::chrono::high_resolution_clock::now(); // Старт CPU таймера
    cpu_process_openmp(cpu.data(), N);            // Обработка массива на CPU
    auto c1 = std::chrono::high_resolution_clock::now(); // Стоп CPU таймера
    double cpu_ms = std::chrono::duration<double, std::milli>(c1 - c0).count(); // Время CPU

    std::vector<int> gpu = base;                  // Копия массива для GPU
    float gpu_ms = gpu_total_ms(gpu.data(), N, threads); // GPU общее время

    int* pinned = nullptr;                        // Указатель на pinned-память
    CHECK_CUDA(cudaHostAlloc(&pinned,
                             N * sizeof(int),
                             cudaHostAllocDefault)); // Выделяем pinned-память
    for (int i = 0; i < N; ++i) pinned[i] = base[i]; // Копируем данные

    double hybrid_ms = hybrid_total_ms(pinned, N, threads); // Гибридная обработка
    CHECK_CUDA(cudaFreeHost(pinned));        // Освобождаем pinned-память

    printf("CPU(OpenMP): %.3f ms\n", cpu_ms);       // Вывод времени CPU
    printf("GPU(total):  %.3f ms\n", gpu_ms);       // Вывод времени GPU
    printf("HYBRID:      %.3f ms\n\n", hybrid_ms);  // Вывод времени гибрида

    printf("Ускорение GPU(total) относительно CPU: %.2f x\n",
           cpu_ms / gpu_ms);                         // Коэффициент ускорения GPU
    printf("Ускорение HYBRID относительно CPU:     %.2f x\n",
           cpu_ms / hybrid_ms);                      // Коэффициент ускорения гибрида

    return 0;                                        // Завершение программы
}

Writing task4.cu


In [12]:
!nvcc -O3 -std=c++17 -Xcompiler -fopenmp task4.cu -o task4 -gencode arch=compute_75,code=sm_75
!./task4

OpenMP включен, потоков (макс): 2
N=1000000

CPU(OpenMP): 0.302 ms
GPU(total):  2.103 ms
HYBRID:      0.896 ms

Ускорение GPU(total) относительно CPU: 0.14 x
Ускорение HYBRID относительно CPU:     0.34 x


**Анализ результатов:**

По результатам сравнения в четвертом задании видно, что при размере массива 1 000 000 элементов быстрее всего выполняется обработка на CPU с OpenMP (0.302 мс). Несмотря на то, что вычисления на GPU сами по себе выполняются очень быстро, общее время GPU-режима оказалось значительно больше (2.103 мс), потому что основную долю занимают копирования данных между хостом и устройством и синхронизация, а сама операция умножения на 2 слишком простая и не успевает “окупать” эти накладные расходы. Гибридный режим показал промежуточный результат (0.896 мс): часть работы действительно выполняется параллельно на CPU и GPU, но затраты на передачу второй половины массива на GPU и возврат результата всё равно остаются, поэтому суммарно он проигрывает чистому CPU. Полученные коэффициенты ускорения меньше единицы (GPU: 0.14×, Hybrid: 0.34×), что означает замедление относительно CPU и подтверждает, что для данной операции и одного прохода по данным наиболее эффективен CPU, а GPU/гибрид становятся выгодными только при более вычислительно сложной обработке или при сценариях, где данные долго остаются на GPU и копирования выполняются редко.